In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
import csv
import time

def scrape_guardian_day(date_obj):
    url = f"https://www.theguardian.com/theguardian/{date_obj.year}/{date_obj.strftime('%b').lower()}/{date_obj.day:02d}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    articles = []

    for link in soup.find_all("a", class_="fc-item__link"):
        headline = link.find("span", class_="js-headline-text")
        if headline:
            articles.append({
                "date": date_obj.strftime("%Y-%m-%d"),
                "headline": headline.text.strip(),
                "url": link["href"]
            })

    return articles

# Define date range for 2024
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2024, 12, 31)

# Write to CSV
with open("guardian_2024_articles.csv", "w", newline='', encoding='utf-8') as csvfile:
    fieldnames = ["date", "headline", "url"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    current_date = start_date
    while current_date <= end_date:
        try:
            articles = scrape_guardian_day(current_date)
            for article in articles:
                writer.writerow(article)
            print(f"Scraped: {current_date} ({len(articles)} articles)")
        except Exception as e:
            print(f"Error on {current_date}: {e}")

        current_date += datetime.timedelta(days=1)
        time.sleep(1)  # Respectful scraping delay

print("Finished scraping The Guardian 2024 archive.")


In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Only run once
nltk.download("vader_lexicon")

# Load the scraped headlines
df = pd.read_csv("guardian_2024_articles.csv")

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis
def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return pd.Series([
        scores['neg'],
        scores['neu'],
        scores['pos'],
        scores['compound']
    ])

# Add new columns to DataFrame
df[['neg', 'neu', 'pos', 'compound']] = df['headline'].apply(analyze_sentiment)

# Save the result
df.to_csv("guardian_2024_sentiment.csv", index=False)

print("Sentiment analysis completed and saved to 'guardian_2024_sentiment.csv'.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data with sentiment scores
df = pd.read_csv("guardian_2024_sentiment.csv")

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Group by date, compute average sentiment
daily_sentiment = df.groupby('date').mean(numeric_only=True)[['neg', 'neu', 'pos', 'compound']]

# Plotting
plt.figure(figsize=(14, 6))
plt.plot(daily_sentiment.index, daily_sentiment['compound'], label='Compound Sentiment', color='purple')
plt.axhline(0, linestyle='--', color='gray')
plt.title("Daily Average Sentiment of Guardian Headlines (2024)")
plt.xlabel("Date")
plt.ylabel("Compound Sentiment Score")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Load full dataset
df = pd.read_csv("guardian_2024_sentiment.csv")

# Example topics: politics, climate, war, tech
topics = ['politics', 'climate', 'environment', 'tech', 'war', 'election']

# Add a new column for detected topic
def detect_topic(url):
    for topic in topics:
        if topic in url.lower():
            return topic
    return 'other'

df['topic'] = df['url'].apply(detect_topic)

# Save filtered version
df.to_csv("guardian_2024_sentiment_with_topics.csv", index=False)


In [ ]:
# Average sentiment per topic
topic_sentiment = df.groupby('topic').mean(numeric_only=True)[['compound']]
print(topic_sentiment.sort_values(by='compound'))


In [ ]:
# Top 5 most negative days
neg_days = df.groupby('date')['compound'].mean().sort_values().head(5)

# Top 5 most positive days
pos_days = df.groupby('date')['compound'].mean().sort_values(ascending=False).head(5)

print("Most Negative Days:")
print(neg_days)

print("\nMost Positive Days:")
print(pos_days)


In [ ]:
pip install wordcloud


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Concatenate all headlines
all_text = " ".join(df['headline'].dropna().astype(str).tolist())

# Generate WordCloud
wordcloud = WordCloud(width=1600, height=800, background_color='white', collocations=False).generate(all_text)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Most Common Words in 2024 Headlines", fontsize=20)
plt.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd # Ensure pandas is imported for df


def show_topic_wordcloud(topic):
    """Generates and displays a word cloud for headlines of a specific topic."""
    # Filter headlines by topic
    topic_df = df[df['topic'] == topic]

    if topic_df.empty:
        print(f"No headlines found for topic: {topic}")
        return

    # Concatenate all headlines for the selected topic
    all_text_topic = " ".join(topic_df['headline'].dropna().astype(str).tolist())

    if not all_text_topic:
        print(f"No text found for word cloud for topic: {topic}")
        return

    # Generate WordCloud
    wordcloud_topic = WordCloud(width=1600, height=800, background_color='white', collocations=False).generate(all_text_topic)

    plt.figure(figsize=(15, 7))
    plt.imshow(wordcloud_topic, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Most Common Words in {topic.capitalize()} Headlines (2024)", fontsize=20)
    plt.show()

topics = df['topic'].unique().tolist()

widgets.interact(show_topic_wordcloud, topic=widgets.Dropdown(options=topics, value='politics'))

In [ ]:
wordcloud.to_file("guardian_wordcloud_2024.png")


In [ ]:
daily_sentiment = df.groupby('date')['compound'].mean()

print("📉 Most Negative Days:")
print(daily_sentiment.sort_values().head(5))

print("\n📈 Most Positive Days:")
print(daily_sentiment.sort_values(ascending=False).head(5))


In [ ]:
summary_df = df.groupby('topic')[['neg', 'neu', 'pos', 'compound']].mean().sort_values(by='compound')
summary_df.to_csv("guardian_2024_topic_sentiment_summary.csv")
summary_df


In [ ]:
topic_trend = df.groupby(['date', 'topic']).size().unstack(fill_value=0)
topic_trend.rolling(window=7).mean().plot(figsize=(16, 6), title="Topic Frequency Over Time (7-day avg)")


In [ ]:
pip install nrclex


In [ ]:
from nrclex import NRCLex
import pandas as pd 
import nltk 


try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("NLTK 'punkt_tab' not found. Downloading...")
    nltk.download('punkt_tab')


def get_emotions(text):
    # Get raw emotion scores
    raw_emotions = NRCLex(text).raw_emotion_scores

    # Define the expected emotion columns
    expected_emotions = ['anger', 'fear', 'joy', 'sadness', 'trust']

    # Create a dictionary with expected emotions, defaulting to 0 if not present
    emotions = {emotion: raw_emotions.get(emotion, 0) for emotion in expected_emotions}

    # Return a Series with the expected columns in a consistent order
    return pd.Series(emotions, index=expected_emotions)

# Apply the function to the 'headline' column
df[['anger', 'fear', 'joy', 'sadness', 'trust']] = df['headline'].apply(get_emotions)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    people = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return pd.Series([people, orgs])

df[['people', 'organizations']] = df['headline'].apply(extract_entities)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['headline'])

kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X)


In [ ]:
import matplotlib.pyplot as plt

df[['anger', 'fear', 'joy', 'sadness', 'trust']].mean().sort_values(ascending=False).plot(
    kind='bar', figsize=(8, 4), color='tomato', title="Average Emotion Scores Across All Headlines"
)
plt.xticks(rotation=45)
plt.ylabel("Average Score")
plt.tight_layout()
plt.show()


In [ ]:
# Show headlines with their recognized entities
df[['headline', 'people', 'organizations']].dropna().head(10)


In [ ]:
from collections import Counter

# Flatten lists of names
people = Counter([name for sublist in df['people'].dropna() for name in sublist])
orgs = Counter([org for sublist in df['organizations'].dropna() for org in sublist])

# Show most common
print("Top People:", people.most_common(5))
print("Top Organizations:", orgs.most_common(5))


In [ ]:
# Show 2 headlines per cluster
for i in range(5):
    print(f"\nCluster {i} Headlines:")
    sample = df[df['cluster'] == i]['headline'].head(2).tolist()
    for headline in sample:
        print("•", headline)


In [ ]:
df['cluster'].value_counts().sort_index()


In [ ]:
import numpy as np

# Get feature names (words from TF-IDF)
words = vectorizer.get_feature_names_out()

# For each cluster, get top N keywords
def get_top_keywords(model, words, n=10):
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    top_words = {}
    for i in range(model.n_clusters):
        top = [words[ind] for ind in order_centroids[i, :n]]
        top_words[i] = top
    return top_words

top_keywords = get_top_keywords(kmeans, words)


In [ ]:
# View top keywords per cluster
for i in range(kmeans.n_clusters):
    print(f"\n📚 Cluster {i}: {', '.join(top_keywords[i])}")
    sample = df[df['cluster'] == i]['headline'].head(3).tolist()
    for h in sample:
        print("   •", h)


In [ ]:
cluster_summary = df.groupby('cluster').agg({
    'headline': 'count',
    'compound': 'mean'
}).rename(columns={'headline': 'count', 'compound': 'avg_sentiment'})

# Add top words
cluster_summary['top_keywords'] = cluster_summary.index.map(lambda i: ", ".join(top_keywords[i]))
cluster_summary


In [ ]:
import matplotlib.pyplot as plt

cluster_summary.sort_values('avg_sentiment')['avg_sentiment'].plot(kind='barh', figsize=(10, 5), color='seagreen')
plt.title("📊 Average Sentiment per Headline Cluster")
plt.xlabel("Sentiment Score")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a smaller sample for better readability
sample_df = df.groupby('cluster').apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)

# Sort by cluster
sample_df = sample_df.sort_values(['cluster', 'compound'])

# Set color palette based on sentiment
def get_sentiment_color(score):
    if score > 0.2:
        return 'green'
    elif score < -0.2:
        return 'red'
    else:
        return 'gray'

sample_df['color'] = sample_df['compound'].apply(get_sentiment_color)

# Plot
plt.figure(figsize=(10, 6))
for idx, row in sample_df.iterrows():
    plt.text(row['cluster'], idx, row['headline'], color=row['color'], fontsize=9)

plt.yticks([])
plt.xlabel("Cluster")
plt.title("📰 Headlines by Cluster and Sentiment Color")
plt.grid(True, axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
!pip install umap-learn


In [ ]:
import umap.umap_ as umap

# Reduce high-dimensional TF-IDF vectors to 2D
reducer = umap.UMAP(random_state=42)
X_umap = reducer.fit_transform(X)

# Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_umap[:, 0], y=X_umap[:, 1], hue=df['cluster'], palette='tab10', s=50)
plt.title("🧭 Headline Clusters Visualized in 2D (UMAP)")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Prepare daily sentiment
df['date'] = pd.to_datetime(df['date'])  # ensure date format
daily_sentiment = df.groupby('date')['compound'].mean().reset_index()

# Step 2: Create lag feature (yesterday's sentiment)
daily_sentiment['prev_day_sentiment'] = daily_sentiment['compound'].shift(1)

# Drop first row (NaN)
forecast_df = daily_sentiment.dropna()

# Step 3: Train a regression model
from sklearn.linear_model import LinearRegression

X = forecast_df[['prev_day_sentiment']]
y = forecast_df['compound']

model = LinearRegression()
model.fit(X, y)

# Step 4: Predict and plot
y_pred = model.predict(X)

import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(forecast_df['date'], y, label='Actual')
plt.plot(forecast_df['date'], y_pred, label='Predicted', linestyle='--')
plt.title("📈 Sentiment Forecasting Using Previous Day")
plt.xlabel("Date")
plt.ylabel("Sentiment")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(df['headline'], df['cluster'], test_size=0.2, random_state=42)

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=3000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train model
clf = LogisticRegression(max_iter=200)
clf.fit(X_train_tfidf, y_train)

# Evaluate
y_pred = clf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


In [ ]:
sample = ["India launches new space mission"]
sample_tfidf = vectorizer.transform(sample)
print("Predicted Cluster:", clf.predict(sample_tfidf)[0])


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import umap.umap_ as umap
import os

# ---------- CONFIG ----------
CSV_PATH = "guardian_2024_sentiment_with_topics.csv"
OUTPUT_PATH = "guardian_2024_sentiment_with_clusters.csv"
N_CLUSTERS = 5
MAX_FEATURES = 3000  

# ---------- LOAD DATA ----------
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File '{CSV_PATH}' not found. Ensure previous steps have been completed.")

df = pd.read_csv(CSV_PATH)
if 'headline' not in df.columns:
    raise ValueError("Missing 'headline' column in the dataset.")

# ---------- TF-IDF VECTORIZE ----------
vectorizer = TfidfVectorizer(stop_words='english', max_features=MAX_FEATURES)
X = vectorizer.fit_transform(df['headline'])

# ---------- CLUSTER ----------
if X.shape[0] < N_CLUSTERS:
    print(f"⚠️ Too few samples ({X.shape[0]}) for {N_CLUSTERS} clusters. Assigning all to cluster 0.")
    df['cluster'] = 0
else:
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init='auto')
    df['cluster'] = kmeans.fit_predict(X)
    print(f"✅ Clustering complete. {N_CLUSTERS} clusters assigned.")

# ---------- SAVE OUTPUT ----------
df.to_csv(OUTPUT_PATH, index=False)
print(f"💾 Saved clustered data to '{OUTPUT_PATH}'")

# ---------- VISUALIZATION ----------
def plot_umap(X, labels):
    reducer = umap.UMAP(random_state=42)
    X_umap = reducer.fit_transform(X)

    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X_umap[:, 0], y=X_umap[:, 1], hue=labels, palette='tab10', s=50)
    plt.title("🧭 Headline Clusters in 2D Space (UMAP)", fontsize=14)
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

plot_umap(X, df['cluster'])

# ---------- Elbow Method to Choose K ----------
def plot_elbow(X, max_k=10):
    distortions = []
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)
    plt.plot(range(2, max_k + 1), distortions, marker='o')
    plt.title("Elbow Method for Optimal K")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Inertia")
    plt.show()

plot_elbow(X)

In [ ]:
import numpy as np

def get_top_keywords_per_cluster(tfidf_vectorizer, kmeans_model, n_keywords=10):
    keywords = np.array(tfidf_vectorizer.get_feature_names_out())
    order_centroids = kmeans_model.cluster_centers_.argsort()[:, ::-1]

    cluster_keywords = {}
    for cluster_num in range(kmeans_model.n_clusters):
        top_terms = keywords[order_centroids[cluster_num, :n_keywords]]
        cluster_keywords[cluster_num] = top_terms.tolist()
    return cluster_keywords

cluster_keywords = get_top_keywords_per_cluster(vectorizer, kmeans)

for cluster, keywords in cluster_keywords.items():
    print(f"Cluster {cluster}: {', '.join(keywords)}")


In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(x='cluster', y='compound', data=df)
plt.title("Sentiment Distribution per Cluster")
plt.xlabel("Cluster")
plt.ylabel("Compound Sentiment Score")
plt.show()


In [ ]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

def extract_entities(texts, label):
    all_entities = []
    for doc in nlp.pipe(texts, disable=["tagger", "parser"]):
        all_entities.extend([ent.text for ent in doc.ents if ent.label_ == label])
    return Counter(all_entities).most_common(10)

for cluster_num in range(N_CLUSTERS):
    headlines = df[df['cluster'] == cluster_num]['headline'].dropna().tolist()
    top_people = extract_entities(headlines, "PERSON")
    top_orgs = extract_entities(headlines, "ORG")
    top_gpe = extract_entities(headlines, "GPE")  

    print(f"\nCluster {cluster_num} Entities:")
    print("Top People:", [p[0] for p in top_people])
    print("Top Organizations:", [o[0] for o in top_orgs])
    print("Top Locations:", [g[0] for g in top_gpe])


In [ ]:
df['date'] = pd.to_datetime(df['date'])  

cluster_date_counts = df.groupby(['date', 'cluster']).size().reset_index(name='count')

plt.figure(figsize=(12, 7))
for cluster_num in range(N_CLUSTERS):
    subset = cluster_date_counts[cluster_date_counts['cluster'] == cluster_num]
    plt.plot(subset['date'], subset['count'], label=f'Cluster {cluster_num}')

plt.title("News Volume Trends per Cluster in 2024")
plt.xlabel("Date")
plt.ylabel("Number of Headlines")
plt.legend()
plt.show()
